All code is from the following Kaggle notebook: https://www.kaggle.com/code/cocoyachi/safedriver-xgboost-musthave-ch08

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import save_npz
from bayes_opt import BayesianOptimization

In [2]:
data_path = '../input/'

train = pd.read_csv(data_path + 'train.csv.zip', index_col='id')
test = pd.read_csv(data_path + 'test.csv.zip', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv.zip', index_col='id')

In [3]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) 

all_features = all_data.columns

In [4]:
cat_features = [feature for feature in all_features if 'cat' in feature]

onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

In [5]:
all_data['num_missing'] = (all_data==-1).sum(axis=1)

In [6]:
remaining_features = [feature for feature in all_features
                      if ('cat' not in feature and 'calc' not in feature)]

remaining_features.append('num_missing')

In [7]:
ind_features = [feature for feature in all_features if 'ind' in feature]

is_first_feature = True
for ind_feature in ind_features:
    if is_first_feature:
        all_data['mix_ind'] = all_data[ind_feature].astype(str) + '_'
        is_first_feature = False
    else:
        all_data['mix_ind'] += all_data[ind_feature].astype(str) + '_'

In [8]:
all_data['mix_ind']

0          2_2_5_1_0_0_1_0_0_0_0_0_0_0_11_0_1_0_
1           1_1_7_0_0_0_0_1_0_0_0_0_0_0_3_0_0_1_
2          5_4_9_1_0_0_0_1_0_0_0_0_0_0_12_1_0_0_
3           0_1_2_0_0_1_0_0_0_0_0_0_0_0_8_1_0_0_
4           0_2_0_1_0_1_0_0_0_0_0_0_0_0_9_1_0_0_
                           ...                  
1488023     0_1_6_0_0_0_1_0_0_0_0_0_0_0_2_0_0_1_
1488024    5_3_5_1_0_0_0_1_0_0_0_0_0_0_11_1_0_0_
1488025     0_1_5_0_0_1_0_0_0_0_0_0_0_0_5_0_0_1_
1488026    6_1_5_1_0_0_0_0_1_0_0_0_0_0_13_1_0_0_
1488027    7_1_4_1_0_0_0_0_1_0_0_0_0_0_12_1_0_0_
Name: mix_ind, Length: 1488028, dtype: object

In [9]:
cat_count_features = []
for feature in cat_features+['mix_ind']:
    val_counts_dict = all_data[feature].value_counts().to_dict()
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x:
                                                           val_counts_dict[x])
    cat_count_features.append(f'{feature}_count')

In [10]:
from scipy import sparse

drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin',
                 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

all_data_remaining = all_data[remaining_features+cat_count_features].drop(drop_features, axis=1)

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),
                               encoded_cat_matrix],
                              format='csr')

In [11]:
num_train = len(train) 


X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

In [12]:
X.shape

(595212, 217)

In [13]:
def eval_gini(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    
    n_samples = y_true.shape[0]
    L_mid = np.linspace(1 / n_samples, 1, n_samples)

    pred_order = y_true[y_pred.argsort()]
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)
    G_pred = np.sum(L_mid - L_pred)

    true_order = y_true[y_true.argsort()]
    L_true = np.cumsum(true_order) / np.sum(true_order)
    G_true = np.sum(L_mid - L_true)
    
    return G_pred / G_true

In [14]:
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds)

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      test_size=0.2,
                                                      random_state=0)

bayes_dtrain = xgb.DMatrix(X_train, y_train)
bayes_dvalid = xgb.DMatrix(X_valid, y_valid)

In [16]:
param_bounds = {'max_depth': (4,12),
                'subsample': (0.6, 1.0),
                'colsample_bytree': (0.5, 1.0),
                'min_child_weight': (5, 20),
                'gamma': (3, 11),
                'reg_alpha': (0, 9),
                'reg_lambda': (0.1, 1.5),
                'scale_pos_weight': (1.0, 2.0)}


fixed_params = {'objective' : 'binary:logistic',
                'learning_rate': 0.02,
                'random_state': 1993}

In [17]:
def eval_function(max_depth, subsample, colsample_bytree, min_child_weight,
                 reg_alpha, gamma, reg_lambda, scale_pos_weight):

    params = {'max_depth': int(round(max_depth)),
              'subsample': subsample,
              'colsample_bytree': colsample_bytree,
              'min_child_weight': min_child_weight,
              'gamma': gamma,
              'reg_alpha':reg_alpha,
              'reg_lambda': reg_lambda,
              'scale_pos_weight': scale_pos_weight}

    params.update(fixed_params)
    
    print('hyperparameters :', params)    
        
    # XGBoost model training
    xgb_model = xgb.train(params=params, 
                          dtrain=bayes_dtrain,
                          num_boost_round=2000,
                          evals=[(bayes_dvalid, 'bayes_dvalid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=False)
                           
    best_iter = xgb_model.best_iteration # optimal number of iterations
    # Make predictions with validation data
    preds = xgb_model.predict(bayes_dvalid, 
                              iteration_range=(0, best_iter))
    # Gini coefficient calculation
    gini_score = eval_gini(y_valid, preds)
    print(f'Gini coefficient: {gini_score}\n')
    
    return gini_score


In [18]:
%%time
optimizer = BayesianOptimization(f=eval_function, 
                                 pbounds=param_bounds, 
                                 random_state=0)


optimizer.maximize(init_points=6, n_iter=50)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------
hyperparameters : {'max_depth': 9, 'subsample': 0.9567092003128319, 'colsample_bytree': 0.7744067519636624, 'min_child_weight': 13.173247744953454, 'gamma': 8.721514930979357, 'reg_alpha': 3.8128931940501425, 'reg_lambda': 1.0042517582933186, 'scale_pos_weight': 1.4375872112626924, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1993}


/home/tunguz/.local/lib/python3.11/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


Gini coefficient: 0.2791697037080871

| 1         | 0.2792    | 0.7744    | 8.722     | 8.822     | 13.17     | 3.813     | 1.004     | 1.438     | 0.9567    |
hyperparameters : {'max_depth': 10, 'subsample': 0.6348517198806163, 'colsample_bytree': 0.9818313802505146, 'min_child_weight': 12.933423796293567, 'gamma': 6.067532150606222, 'reg_alpha': 5.112401049845391, 'reg_lambda': 1.3958352936097254, 'scale_pos_weight': 1.0710360581978868, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1993}
Gini coefficient: 0.28113448946604874

| 2         | 0.2811    | 0.9818    | 6.068     | 10.33     | 12.93     | 5.112     | 1.396     | 1.071     | 0.6349    |
hyperparameters : {'max_depth': 10, 'subsample': 0.9122116705145822, 'colsample_bytree': 0.5101091987201629, 'min_child_weight': 18.050182223702286, 'gamma': 9.660958764383505, 'reg_alpha': 8.807565080094877, 'reg_lambda': 1.218821989903413, 'scale_pos_weight': 1.4614793622529318, 'objective': 'binary:logistic', 'lear

/home/tunguz/.local/lib/python3.11/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


Gini coefficient: 0.2831538435080924

| 7         | 0.2832    | 1.0       | 3.0       | 7.963     | 9.678     | 5.987     | 1.5       | 1.165     | 0.6       |
hyperparameters : {'max_depth': 8, 'subsample': 0.8159658985370651, 'colsample_bytree': 0.6816097525697367, 'min_child_weight': 9.393673977936512, 'gamma': 3.0038029128494568, 'reg_alpha': 6.350105057926053, 'reg_lambda': 0.7622613329095277, 'scale_pos_weight': 1.3034592558542544, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1993}


...

Gini coefficient: 0.2817289784712896

| 54        | 0.2817    | 0.8996    | 3.116     | 5.553     | 6.238     | 8.843     | 0.4663    | 1.054     | 0.8872    |
hyperparameters : {'max_depth': 6, 'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 11.81519748448993, 'gamma': 4.2428371517554, 'reg_alpha': 2.615400423173235, 'reg_lambda': 0.1, 'scale_pos_weight': 1.0, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1993}


/home/tunguz/.local/lib/python3.11/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


Gini coefficient: 0.28358669798057246

| 55        | 0.2836    | 0.5       | 4.243     | 5.952     | 11.82     | 2.615     | 0.1       | 1.0       | 0.6       |
hyperparameters : {'max_depth': 4, 'subsample': 0.6, 'colsample_bytree': 0.5, 'min_child_weight': 11.419413476751902, 'gamma': 3.0, 'reg_alpha': 7.224648542076648, 'reg_lambda': 0.2955600803585693, 'scale_pos_weight': 2.0, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1993}


/home/tunguz/.local/lib/python3.11/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


Gini coefficient: 0.28562446250038004

| 56        | 0.2856    | 0.5       | 3.0       | 4.0       | 11.42     | 7.225     | 0.2956    | 2.0       | 0.6       |
CPU times: user 1d 12h 50min 9s, sys: 2min 59s, total: 1d 12h 53min 9s
Wall time: 2h 29min 16s


In [20]:
max_params = optimizer.max['params']
max_params

{'colsample_bytree': 0.5,
 'gamma': 3.0,
 'max_depth': 6.7298867719391895,
 'min_child_weight': 5.0,
 'reg_alpha': 5.754680072558848,
 'reg_lambda': 0.1,
 'scale_pos_weight': 1.0,
 'subsample': 0.6}

In [21]:
max_params['max_depth'] = int(round(max_params['max_depth']))

max_params.update(fixed_params)
max_params

{'colsample_bytree': 0.5,
 'gamma': 3.0,
 'max_depth': 7,
 'min_child_weight': 5.0,
 'reg_alpha': 5.754680072558848,
 'reg_lambda': 0.1,
 'scale_pos_weight': 1.0,
 'subsample': 0.6,
 'objective': 'binary:logistic',
 'learning_rate': 0.02,
 'random_state': 1993}

In [22]:
%%time

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)


oof_val_preds = np.zeros(X.shape[0]) 
oof_test_preds = np.zeros(X_test.shape[0]) 
dtest = xgb.DMatrix(X_test)

for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):

    print('#'*40, f'Fold {idx+1} / {folds.n_splits}', '#'*40)
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]


    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_valid, y_valid)
   

    xgb_model = xgb.train(params=max_params, 
                          dtrain=dtrain,
                          num_boost_round=2000,
                          evals=[(dvalid, 'valid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=100)

    best_iter = xgb_model.best_iteration
    oof_test_preds += xgb_model.predict(dtest,
                                        iteration_range=(0, best_iter))/folds.n_splits
    
    oof_val_preds[valid_idx] += xgb_model.predict(dvalid, 
                                                  iteration_range=(0, best_iter))
    
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'Fold {idx+1} gini score : {gini_score}\n')

######################################## Fold 1 / 5 ########################################


/home/tunguz/.local/lib/python3.11/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	valid-logloss:0.21137	valid-gini:0.19231
[100]	valid-logloss:0.15529	valid-gini:0.27176
[200]	valid-logloss:0.15202	valid-gini:0.28596
[300]	valid-logloss:0.15163	valid-gini:0.29233
[400]	valid-logloss:0.15147	valid-gini:0.29643
[500]	valid-logloss:0.15143	valid-gini:0.29755
[600]	valid-logloss:0.15141	valid-gini:0.29791
[700]	valid-logloss:0.15140	valid-gini:0.29831
[800]	valid-logloss:0.15139	valid-gini:0.29865
[900]	valid-logloss:0.15138	valid-gini:0.29891
[1000]	valid-logloss:0.15138	valid-gini:0.29893
[1100]	valid-logloss:0.15137	valid-gini:0.29916
[1200]	valid-logloss:0.15138	valid-gini:0.29872
[1300]	valid-logloss:0.15139	valid-gini:0.29879
[1315]	valid-logloss:0.15138	valid-gini:0.29890
Fold 1 gini score : 0.2991940038752933

######################################## Fold 2 / 5 ########################################
[0]	valid-logloss:0.21137	valid-gini:0.18049
[100]	valid-logloss:0.15543	valid-gini:0.26108
[200]	valid-logloss:0.15227	valid-gini:0.27386
[300]	valid-logloss:

In [23]:
print('OOF Verification data Gini coefficient:', eval_gini(y, oof_val_preds))


OOF Verification data Gini coefficient: 0.28864889403014304


In [24]:
submission['target'] = oof_test_preds
submission.to_csv('../submissions/submission_best_kaggle_xgb_2.csv')

In [27]:
fixed_params = {'objective' : 'binary:logistic',
                'learning_rate': 0.005,
                'random_state': 1993}

max_params.update(fixed_params)
max_params

{'colsample_bytree': 0.5,
 'gamma': 3.0,
 'max_depth': 7,
 'min_child_weight': 5.0,
 'reg_alpha': 5.754680072558848,
 'reg_lambda': 0.1,
 'scale_pos_weight': 1.0,
 'subsample': 0.6,
 'objective': 'binary:logistic',
 'learning_rate': 0.005,
 'random_state': 1993}

In [28]:
%%time

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)


oof_val_preds = np.zeros(X.shape[0]) 
oof_test_preds = np.zeros(X_test.shape[0]) 
dtest = xgb.DMatrix(X_test)

for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):

    print('#'*40, f'Fold {idx+1} / {folds.n_splits}', '#'*40)
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]


    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_valid, y_valid)
   

    xgb_model = xgb.train(params=max_params, 
                          dtrain=dtrain,
                          num_boost_round=20000,
                          evals=[(dvalid, 'valid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=1000,
                          verbose_eval=400)

    best_iter = xgb_model.best_iteration
    oof_test_preds += xgb_model.predict(dtest,
                                        iteration_range=(0, best_iter))/folds.n_splits
    
    oof_val_preds[valid_idx] += xgb_model.predict(dvalid, 
                                                  iteration_range=(0, best_iter))
    
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'Fold {idx+1} gini score : {gini_score}\n')

######################################## Fold 1 / 5 ########################################


/home/tunguz/.local/lib/python3.11/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	valid-logloss:0.21263	valid-gini:0.19231
[400]	valid-logloss:0.15541	valid-gini:0.27204
[800]	valid-logloss:0.15202	valid-gini:0.28597
[1200]	valid-logloss:0.15160	valid-gini:0.29323
[1600]	valid-logloss:0.15145	valid-gini:0.29684
[2000]	valid-logloss:0.15140	valid-gini:0.29823
[2400]	valid-logloss:0.15137	valid-gini:0.29896
[2800]	valid-logloss:0.15135	valid-gini:0.29966
[3200]	valid-logloss:0.15133	valid-gini:0.30010
[3600]	valid-logloss:0.15132	valid-gini:0.30029
[4000]	valid-logloss:0.15131	valid-gini:0.30054
[4400]	valid-logloss:0.15131	valid-gini:0.30071
[4800]	valid-logloss:0.15131	valid-gini:0.30086
[5200]	valid-logloss:0.15130	valid-gini:0.30087
[5600]	valid-logloss:0.15130	valid-gini:0.30094
[6000]	valid-logloss:0.15130	valid-gini:0.30096
[6400]	valid-logloss:0.15130	valid-gini:0.30099
[6800]	valid-logloss:0.15130	valid-gini:0.30096
[7200]	valid-logloss:0.15130	valid-gini:0.30100
[7600]	valid-logloss:0.15130	valid-gini:0.30104
[8000]	valid-logloss:0.15130	valid-gini:0.301

In [29]:
print('OOF Verification data Gini coefficient:', eval_gini(y, oof_val_preds))

OOF Verification data Gini coefficient: 0.2897355723149737


In [30]:
submission['target'] = oof_test_preds
submission.to_csv('../submissions/submission_best_kaggle_xgb_3.csv')

In [31]:
max_params_0 = {'colsample_bytree': 0.92975858050776,
 'gamma': 9.95563546750357,
 'max_depth': 7,
 'min_child_weight': 6.249564429359247,
 'reg_alpha': 8.411512219837842,
 'reg_lambda': 1.424460008293778,
 'scale_pos_weight': 1.5416807226581535,
 'subsample': 0.8535233675350644,
 'objective': 'binary:logistic',
 'learning_rate': 0.01,
 'random_state': 1993}

In [32]:
%%time

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)


oof_val_preds = np.zeros(X.shape[0]) 
oof_test_preds = np.zeros(X_test.shape[0]) 
dtest = xgb.DMatrix(X_test)

for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):

    print('#'*40, f'Fold {idx+1} / {folds.n_splits}', '#'*40)
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]


    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_valid, y_valid)
   

    xgb_model = xgb.train(params=max_params_0, 
                          dtrain=dtrain,
                          num_boost_round=10000,
                          evals=[(dvalid, 'valid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=200)

    best_iter = xgb_model.best_iteration
    oof_test_preds += xgb_model.predict(dtest,
                                        iteration_range=(0, best_iter))/folds.n_splits
    
    oof_val_preds[valid_idx] += xgb_model.predict(dvalid, 
                                                  iteration_range=(0, best_iter))
    
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'Fold {idx+1} gini score : {gini_score}\n')

######################################## Fold 1 / 5 ########################################


/home/tunguz/.local/lib/python3.11/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	valid-logloss:0.21996	valid-gini:0.20901
[200]	valid-logloss:0.16179	valid-gini:0.27455
[400]	valid-logloss:0.15622	valid-gini:0.28722
[600]	valid-logloss:0.15546	valid-gini:0.29128
[800]	valid-logloss:0.15535	valid-gini:0.29186
[1000]	valid-logloss:0.15533	valid-gini:0.29202
[1200]	valid-logloss:0.15532	valid-gini:0.29232
[1400]	valid-logloss:0.15532	valid-gini:0.29240
[1600]	valid-logloss:0.15530	valid-gini:0.29252
[1800]	valid-logloss:0.15531	valid-gini:0.29254
[2000]	valid-logloss:0.15531	valid-gini:0.29265
[2200]	valid-logloss:0.15530	valid-gini:0.29272
[2366]	valid-logloss:0.15530	valid-gini:0.29273
Fold 1 gini score : 0.2927066786908154

######################################## Fold 2 / 5 ########################################
[0]	valid-logloss:0.21996	valid-gini:0.19909
[200]	valid-logloss:0.16203	valid-gini:0.26142
[400]	valid-logloss:0.15664	valid-gini:0.27163
[600]	valid-logloss:0.15591	valid-gini:0.27571
[800]	valid-logloss:0.15581	valid-gini:0.27639
[1000]	valid-logl

In [42]:
save_npz('../input/X_test_217.npz', X_test)

In [43]:
np.save('../input/y_217.npy', y)